In [ ]:
import os
import cv2
import numpy as np
# Encoding and Split data into Train/Test Sets
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
# Tensorflow Keras CNN Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
# Plot Images
import matplotlib.pyplot as plt

In [ ]:
folder_dir = './data'

In [ ]:
data_dir = './data'
datagen = ImageDataGenerator()


class_names = list(datagen.flow_from_directory(data_dir).class_indices.keys())

# Print the flower names
print("Flower names in the data folder:")
for flower in class_names:
    print(flower)

In [ ]:
count = 0 
dirs = os.listdir('data/')
for dir in dirs:
    files = list(os.listdir('data/'+dir))
    print( dir +' Folder has '+ str(len(files)) + ' images')
    count = count + len(files)
print( 'data Folder has '+ str(count) + ' images')

In [ ]:
data_dir = './data'
datagen = ImageDataGenerator(rescale=1./255)
train_ds = datagen.flow_from_directory(
    data_dir,
)
images, labels = next(train_ds)
flower_names = list(train_ds.class_indices.keys())

plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(images[i])
    plt.title(flower_names[int(labels[i].argmax())])
    plt.axis('off')

plt.show()

In [ ]:
data = []
label = []

SIZE = 128  # Crop the image to 128x128

for folder in os.listdir(folder_dir):
    for file in os.listdir(os.path.join(folder_dir, folder)):
        if file.endswith("jpg"):
            label.append(folder)
            img = cv2.imread(os.path.join(folder_dir, folder, file))
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            im = cv2.resize(img_rgb, (SIZE, SIZE))
            data.append(im)
        else:
            continue

In [ ]:
data_arr = np.array(data)
label_arr = np.array(label)

In [ ]:
encoder = LabelEncoder()
y = encoder.fit_transform(label_arr)
y = to_categorical(y, 5)
X = data_arr / 255

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)

In [ ]:
from tensorflow.keras.layers import Input

model = Sequential()
model.add(Input(shape=(SIZE, SIZE, 3)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(5, activation="softmax"))

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.20,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    vertical_flip=True)

datagen.fit(X_train)

In [ ]:
from tensorflow.keras.optimizers import Adam

# Compile the model with the correct learning rate argument
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define batch size and epochs
batch_size = 32
epochs =16

# Train the model using the fit method
history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                    epochs=epochs,
                    validation_data=(X_test, y_test),
                    verbose=1)

In [ ]:
categories = np.sort(os.listdir(folder_dir))
fig, ax = plt.subplots(6, 6, figsize=(25, 40))

for i in range(6):
    for j in range(6):
        k = int(np.random.random_sample() * len(X_test))
        if (categories[np.argmax(y_test[k])] == categories[np.argmax(model.predict(X_test)[k])]):
            ax[i, j].set_title("TRUE: " + categories[np.argmax(y_test[k])], color='green')
            ax[i, j].set_xlabel("PREDICTED: " + categories[np.argmax(model.predict(X_test)[k])], color='green')
            ax[i, j].imshow(np.array(X_test)[k].reshape(SIZE, SIZE, 3), cmap='gray')
        else:
            ax[i, j].set_title("TRUE: " + categories[np.argmax(y_test[k])], color='red')
            ax[i, j].set_xlabel("PREDICTED: " + categories[np.argmax(model.predict(X_test)[k])], color='red')
            ax[i, j].imshow(np.array(X_test)[k].reshape(SIZE, SIZE, 3), cmap='gray')